In [116]:
!which python

/Users/apabook/Desktop/ds-deployment/.venv/bin/python


In [117]:
import pandas as pd
import requests
import json
import pprint
from uuid import uuid4

# Complete dataspace flow

## Consts


In [118]:
data_space_provider = "http://127.0.0.1:1234"
data_space_consumer = "http://127.0.0.1:1235"
api_endpoint = "https://jsonplaceholder.typicode.com/posts"

data_space_provider_participant_id = ""
data_space_consumer_participant_id = ""
catalog_id = ""
dataset_id = ""
distribution_id = ""
data_service_id = ""
policy_id = ""
target_id = ""

cn_consumer_id = ""
cn_provider_id = ""
agreement_id = ""
transfer_consumer_id = ""
transfer_provider_id = ""

json_header = {
    "Content-Type": "application/json",
}

## Participants onboarding

In [120]:
url = data_space_provider + "/api/v1/participants"
payload = {
    "dspace:participantType": "Provider",
    "dspace:participantBaseUrl": data_space_provider,
    "dspace:extraFields": {}
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_space_provider_participant_id = response_as_json["participant_id"]
data_space_provider_participant_id

'urn:uuid:0c409422-63b1-4f1a-9adb-8efb20971698'

In [121]:
url = data_space_provider + "/api/v1/participants"
payload = {
    "dspace:participantType": "Consumer",
    "dspace:participantBaseUrl": data_space_consumer,
    "dspace:extraFields": {}
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_space_consumer_participant_id = response_as_json["participant_id"]
data_space_consumer_participant_id

'urn:uuid:82994955-f46c-4371-b661-dbf82cca9d56'

## Create Catalog instances and metadata

In [122]:
url = data_space_provider + "/api/v1/catalogs"
payload = {
    "foaf:homepage": "My dataspace catalog",
    "dct:title": "My dataspace catalog",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]
catalog_id

'urn:uuid:26061cde-f392-4560-97c0-d7465eb1da33'

In [123]:
url = data_space_provider + "/api/v1/catalogs/{}/datasets".format(catalog_id)
payload = {
    "dct:title": "My dataspace dataset",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
dataset_id = response_as_json["@id"]
dataset_id

'urn:uuid:7eed6a02-7f28-4148-b031-02d66e36e7f3'

In [124]:
url = data_space_provider + "/api/v1/catalogs/{}/data-services".format(catalog_id)
payload = {
    "dcat:endpointURL": "https://jsonplaceholder.typicode.com/posts"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_service_id = response_as_json["@id"]
data_service_id

'urn:uuid:b665213a-07e5-464e-9c24-5725e61622a5'

In [125]:
url = data_space_provider + "/api/v1/catalogs/{}/datasets/{}/distributions".format(catalog_id, dataset_id)
payload = {
    "dcat:accessService": data_service_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
dataset_id = response_as_json["@id"]
dataset_id

'urn:uuid:56324666-f63e-40b4-a632-088692a1b8b5'

## Create policies

In [126]:
url = data_space_provider + "/api/v1/datasets/{}/policies".format(dataset_id)
payload = {
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }
    ],
    "obligation": [],
    "prohibition": []
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
policy_id = response_as_json["id"]
target_id = response_as_json["entity"]
print(json.dumps(response_as_json, indent=4))

{
    "id": "urn:uuid:7a763025-16b3-4b5f-82ab-4759787f99f3",
    "odrl_offer": {
        "obligation": [],
        "permission": [
            {
                "action": "use",
                "constraint": [
                    {
                        "leftOperand": "did:web:hola.es",
                        "operator": "eq",
                        "rightOperand": "user"
                    }
                ]
            }
        ],
        "prohibition": []
    },
    "entity": "urn:uuid:56324666-f63e-40b4-a632-088692a1b8b5",
    "entity_type": "Dataset"
}


## Contract Negotiation

In [127]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerAddress": data_space_provider,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))


{
  "consumerPid": "urn:uuid:771f8ecd-419c-4168-a567-ed73146eafe3",
  "providerPid": "urn:uuid:a6fe3287-9880-412c-9000-21f352e52cdb",
  "offer": {
    "@id": "urn:uuid:7a763025-16b3-4b5f-82ab-4759787f99f3",
    "permission": [
      {
        "action": "use"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:56324666-f63e-40b4-a632-088692a1b8b5"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:a6fe3287-9880-412c-9000-21f352e52cdb",
    "consumerPid": "urn:uuid:771f8ecd-419c-4168-a567-ed73146eafe3",
    "state": "REQUESTED"
  }
}


In [128]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerParticipantId": "urn:uuid:82994955-f46c-4371-b661-dbf82cca9d56",
  "consumerPid": "urn:uuid:771f8ecd-419c-4168-a567-ed73146eafe3",
  "providerPid": "urn:uuid:e46116cf-811d-4a3e-b93a-075d1f3f9390",
  "offer": {
    "@id": "urn:uuid:7a763025-16b3-4b5f-82ab-4759787f99f3",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:56324666-f63e-40b4-a632-088692a1b8b5"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:a6fe3287-9880-412c-9000-21f352e52cdb",
    "consumerPid": "urn:uuid:771f8ecd-419c-4168-a567-ed73146eafe3",
    "state": "OFFERED"
  }
}


In [129]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-acceptance"
payload = {
    "providerAddress": data_space_provider,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:771f8ecd-419c-4168-a567-ed73146eafe3",
  "providerPid": "urn:uuid:a6fe3287-9880-412c-9000-21f352e52cdb",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:a6fe3287-9880-412c-9000-21f352e52cdb",
    "consumerPid": "urn:uuid:771f8ecd-419c-4168-a567-ed73146eafe3",
    "state": "ACCEPTED"
  }
}


In [130]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-agreement"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "agreement": {
        "@id": "urn:uuid:{}".format(uuid4()),
        "@type": "Agreement",
        "target": target_id,
        "assigner": data_space_provider_participant_id,
        "assignee": data_space_consumer_participant_id,
        "permission": [
            {
                "action": "supermegause"
            }
        ],
        "timestamp": "2023-02-12T12:00:00"
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))
agreement_id = response_as_json["agreement"]["@id"]

{
  "consumerPid": "urn:uuid:771f8ecd-419c-4168-a567-ed73146eafe3",
  "providerPid": "urn:uuid:a6fe3287-9880-412c-9000-21f352e52cdb",
  "agreement": {
    "@id": "urn:uuid:53f69453-e621-46e7-bdec-5b1bb33bf344",
    "profile": null,
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "obligation": null,
    "@type": "Agreement",
    "target": "urn:uuid:56324666-f63e-40b4-a632-088692a1b8b5",
    "assigner": "urn:uuid:0c409422-63b1-4f1a-9adb-8efb20971698",
    "assignee": "urn:uuid:82994955-f46c-4371-b661-dbf82cca9d56",
    "timestamp": "2023-02-12T12:00:00",
    "prohibition": null
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:a6fe3287-9880-412c-9000-21f352e52cdb",
    "consumerPid": "urn:uuid:771f8ecd-419c-4168-a567-ed73146eafe3",
    "state": "AGREED"
  }
}


In [131]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-verification"
payload = {
    "providerAddress": data_space_provider,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:771f8ecd-419c-4168-a567-ed73146eafe3",
  "providerPid": "urn:uuid:a6fe3287-9880-412c-9000-21f352e52cdb",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:a6fe3287-9880-412c-9000-21f352e52cdb",
    "consumerPid": "urn:uuid:771f8ecd-419c-4168-a567-ed73146eafe3",
    "state": "VERIFIED"
  }
}


In [132]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-finalization"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:771f8ecd-419c-4168-a567-ed73146eafe3",
  "providerPid": "urn:uuid:a6fe3287-9880-412c-9000-21f352e52cdb",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:a6fe3287-9880-412c-9000-21f352e52cdb",
    "consumerPid": "urn:uuid:771f8ecd-419c-4168-a567-ed73146eafe3",
    "state": "FINALIZED"
  }
}


## Transfer Negotiation


In [137]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-request"
payload = {
    "providerAddress": data_space_provider,
    "agreementId": agreement_id,
    "format": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
transfer_consumer_id = response_as_json["consumerPid"]
transfer_provider_id = response_as_json["providerPid"]
callback_address = response_as_json["callbackAddress"]
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:6930a7d9-c530-4c5a-8dea-ad212da22c15",
  "consumerPid": "urn:uuid:708d25fd-8cd1-42f5-beb5-14053653687b",
  "agreementId": "urn:uuid:53f69453-e621-46e7-bdec-5b1bb33bf344",
  "callbackAddress": "http://127.0.0.1:1235/urn:uuid:ade8fb49-95a5-4ec6-9b88-16ccf65a0307",
  "format": "http+pull",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:6930a7d9-c530-4c5a-8dea-ad212da22c15",
    "consumerPid": "urn:uuid:708d25fd-8cd1-42f5-beb5-14053653687b",
    "state": "REQUESTED"
  }
}


In [138]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-start"
payload = {
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
    "dataAddress": {
        "@type": "DataAddress",
        "endpointType": "https://w3id.org/idsa/v4.1/HTTP",
        "endpoint": "http://example.com",
        "endpointProperties": [
            {
                "@type": "EndpointProperty",
                "name": "authorization",
                "value": "TOKEN-ABCDEFG"
            },
            {
                "@type": "EndpointProperty",
                "name": "authType",
                "value": "bearer"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:6930a7d9-c530-4c5a-8dea-ad212da22c15",
  "consumerPid": "urn:uuid:708d25fd-8cd1-42f5-beb5-14053653687b",
  "dataAddress": {
    "@type": "DataAddress",
    "endpointType": "https://w3id.org/idsa/v4.1/HTTP",
    "endpoint": "http://example.com",
    "endpointProperties": [
      {
        "@type": "EndpointProperty",
        "name": "authorization",
        "value": "TOKEN-ABCDEFG"
      },
      {
        "@type": "EndpointProperty",
        "name": "authType",
        "value": "bearer"
      }
    ]
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:6930a7d9-c530-4c5a-8dea-ad212da22c15",
    "consumerPid": "urn:uuid:708d25fd-8cd1-42f5-beb5-14053653687b",
    "state": "STARTED"
  }
}


In [139]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "providerAddress": data_space_provider,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:6930a7d9-c530-4c5a-8dea-ad212da22c15",
  "consumerPid": "urn:uuid:708d25fd-8cd1-42f5-beb5-14053653687b",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:6930a7d9-c530-4c5a-8dea-ad212da22c15",
    "consumerPid": "urn:uuid:708d25fd-8cd1-42f5-beb5-14053653687b",
    "state": "SUSPENDED"
  }
}


In [140]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-start"
payload = {
    "providerAddress": data_space_provider,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:6930a7d9-c530-4c5a-8dea-ad212da22c15",
  "consumerPid": "urn:uuid:708d25fd-8cd1-42f5-beb5-14053653687b",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:6930a7d9-c530-4c5a-8dea-ad212da22c15",
    "consumerPid": "urn:uuid:708d25fd-8cd1-42f5-beb5-14053653687b",
    "state": "STARTED"
  }
}


In [141]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-completion"
payload = {
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:6930a7d9-c530-4c5a-8dea-ad212da22c15",
  "consumerPid": "urn:uuid:708d25fd-8cd1-42f5-beb5-14053653687b",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:6930a7d9-c530-4c5a-8dea-ad212da22c15",
    "consumerPid": "urn:uuid:708d25fd-8cd1-42f5-beb5-14053653687b",
    "state": "COMPLETED"
  }
}
